In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 71.3 MB/s eta 0:00:00


In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

The **climatology** for **pre-monsoon (MAM)** and **monsoon (JJAS)** season was derived from ERA5 **500hpa-200hpa Potential vorticity**  data (1981–2010)

In [ ]:
ds=xr.open_dataset('/content/drive/MyDrive/AP_HW/ERA5 Data/potential_vorticity/pv-2023.nc')
ds

<xarray.Dataset> Size: 3GB
Dimensions:         (valid_time: 2920, pressure_level: 3, latitude: 209,
                     longitude: 381)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2023-01-01 ... 2023-12-3...
  * pressure_level  (pressure_level) float64 24B 500.0 300.0 200.0
  * latitude        (latitude) float64 2kB 52.0 51.75 51.5 ... 0.5 0.25 0.0
  * longitude       (longitude) float64 3kB 25.0 25.25 25.5 ... 119.8 120.0
    expver          (valid_time) <U4 47kB ...
Data variables:
    pv              (valid_time, pressure_level, latitude, longitude) float32 3GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-09-03T09:10 GRIB to CDM+CF via cfgrib-0.9.1...

# MAM


In [ ]:

# @title vertically averaged
import os
from tqdm.auto import tqdm
import datetime
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))  # Print current time (+6h offset)

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/potential_vorticity'

pv_list = []

# Loop through files in directory
for filename in tqdm(sorted(os.listdir(directory_path))):
  if filename.endswith('.nc'):  # Process only netCDF files
    file_path = os.path.join(directory_path, filename)
    year = filename[-7:-3]  # Extract year from filename
    ds_pv = xr.open_dataset(file_path)

    # Define seasonal time range (MAM) and convert to bd time
    start_time = str(np.datetime64(f'{year}-03-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
    end_time = str(np.datetime64(f'{year}-05-31T17:00:00.000000000'))
    time_range = slice(start_time, end_time)

    # Select, adjust to Bangladesh timezone, and average dataset
    ds_pv = ds_pv.sel(valid_time=time_range)
    ds_pv['valid_time'] = ds_pv['valid_time'] + np.timedelta64(6, 'h')
    ds_pv = ds_pv.mean(dim='pressure_level', skipna=True)        # Vertical mean
    ds_pv = ds_pv.groupby('valid_time.year').mean(dim='valid_time', skipna=True)  # Seasonal mean

    pv_list.append(ds_pv)

# Concatenate yearly data
pv_ds = xr.concat(pv_list, dim='year')

# Compute climatology (1981–2010)
pv_clim_ds = pv_ds.sel(year=slice(1981, 2010))
pv_climatology = pv_clim_ds.mean(dim='year', skipna=True)

# Save to NetCDF
pv_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/pv/pv_mam_vertical_climatology.nc')

# Print completion time
import datetime
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 11:38:59.879784


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 12:10:01.972949


# JJAS


In [ ]:
# @title vertically averaged.

import os
from tqdm.auto import tqdm
import datetime
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))  # Print current time (+6h offset)

directory_path = '/content/drive/MyDrive/AP_HW/ERA5 Data/potential_vorticity'

pv_list = []

# Loop through all files
for filename in tqdm(sorted(os.listdir(directory_path))):
  if filename.endswith('.nc'):  # Process only netCDF files
    file_path = os.path.join(directory_path, filename)
    year = filename[-7:-3]  # Extract year
    ds_pv = xr.open_dataset(file_path)

    # Define JJAS time range
    start_time = str(np.datetime64(f'{year}-06-01T00:00:00.000000000') - np.timedelta64(6, 'h'))
    end_time = str(np.datetime64(f'{year}-09-30T17:00:00.000000000'))
    time_range = slice(start_time, end_time)

    # Select time, adjust to Bangladesh timezone, and average
    ds_pv = ds_pv.sel(valid_time=time_range)
    ds_pv['valid_time'] = ds_pv['valid_time'] + np.timedelta64(6, 'h')
    ds_pv = ds_pv.mean(dim='pressure_level', skipna=True)  # Vertical mean
    ds_pv = ds_pv.groupby('valid_time.year').mean(dim='valid_time', skipna=True)  # Seasonal mean

    pv_list.append(ds_pv)

# Merge yearly data
pv_ds = xr.concat(pv_list, dim='year')

# Compute climatology (1981–2010)
pv_clim_ds = pv_ds.sel(year=slice(1981, 2010))
pv_climatology = pv_clim_ds.mean(dim='year', skipna=True)

# Save climatology
pv_climatology.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/pv/pv_jjas_vertical_climatology.nc')

# Print completion time
import datetime
print(datetime.datetime.utcnow() + datetime.timedelta(hours=6))


2025-06-21 12:25:51.962731


  0%|          | 0/54 [00:00<?, ?it/s]

2025-06-21 12:53:37.845528
